In [4]:
path = 'data/'

In [5]:
# Extract images, tables, and chunk text
from unstructured.partition.pdf import partition_pdf

raw_pdf_elements = partition_pdf(
    filename=path + "physics_example.pdf",
    extract_images_in_pdf=True,
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    image_output_dir_path=path,
)

[nltk_data] Downloading package punkt to /Users/cal/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/cal/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [7]:
# Categorize text elements by type
tables = []
texts = []
for element in raw_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        tables.append(str(element))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        texts.append(str(element))

In [9]:
import os
import uuid
import chromadb
import numpy as np
from PIL import Image as _PILImage
from langchain.vectorstores import Chroma
from langchain_experimental.open_clip import OpenCLIPEmbeddings

vectorstore = Chroma(
    collection_name="cyrus_rag_clip", embedding_function=OpenCLIPEmbeddings()
)


image_uris = sorted(
    [
        os.path.join(path, image_name)
        for image_name in os.listdir(path)
        if image_name.endswith(".jpg")
    ]
)

# Add images
vectorstore.add_images(uris=image_uris)

# Add documents
vectorstore.add_texts(texts=texts)

# Make retriever
retriever = vectorstore.as_retriever()

['8.01x Classical Mechanics, Fall 2016\n\nMassachusetts Institute of Technology\n\nProblem Set 1\n\n1. Car and Bicycle Rider\n\nA car is driving along a straight line with a speed v0. At time t = 0 the car is at the origin. At a later instant of time t = t1 the car starts to slow down until it stops at a time t = t2. The acceleration of the car as a function of time is given by\n\na= 0 0<t<t, co —c(t — t1) ty <t<tg\n\n(cid:26)\n\nwhere c is a positive constant which has dimensions of acceleration per unit time.\n\n(a) Find vc(t) and xc(t), the x-component of the velocity and the position of the car as a function of time. Express your answer in terms of some or all of the following variables: v0, c, t, t1 and t2.\n\n(b) A bicycle rider is riding at a constant speed of vb and at t = 0 is 17 m behind the car. The cyclist reaches the car when the car just comes to rest. The car is moving with an acceleration\n\na- 0 0<t<t co —c(t — t1) ty <t<tg\n\n(cid:26)\n\nwhere its initial component of